In [1]:
import shortuuid
import names
import mariadb
import sys
import pandas as pd

In [17]:
# Connect to MariaDB Platform
try:
    conn = mariadb.connect(
        user="root",
        password="AXOYu0mzs8vWyMGTBPLl",
        host='containers-us-west-16.railway.app',
        port=5604,
        database="railway"

    )
except mariadb.Error as e:
    print(f"Error connecting to MariaDB Platform: {e}")
    sys.exit(1)

# Get Cursor
cur = conn.cursor()

In [18]:
cur.execute("SHOW tables;")
rows = cur.fetchall()
df_tables = pd.DataFrame(data=rows, columns=['TABLES'])
df_tables

,TABLES
0,EXPENSE
1,HAS_GROUP
2,USER
3,USER_FRIEND
4,USER_HAS_GROUP_EXPENSE


# INSERT USER

In [19]:
# INSERT USER
for i in range(15):
    id = shortuuid.uuid()
    name = names.get_full_name()
    cur.execute(
    "INSERT INTO USER (user_id, user_name) VALUES (?, ?);", 
    (id, name))

In [20]:
cur.execute("SELECT * FROM USER;")
rows = cur.fetchall()
df_user = pd.DataFrame(data=rows, columns=['user_id', 'user_name'])
df_user

,user_id,user_name
0,3xyaufSzzUp9LPkSKTxhqz,Jacob Allen
1,5jZkqyec54QYyaqgp4h47u,Dennis Watson
2,5qqCExUuQ5drJaYt7F3B5K,Darrell Elliott
3,7j28XCkCHvtGu3Bqns8TTh,Kelly Budde
4,AYn937uXRJ63ehbPFSmbLR,Joseph Farnan
5,bMmeWsNy5JZAtf8WuaF9oC,Bobby Barr
6,CbE6U4QrKJTcnUCMnW5FiZ,Peter Stansberry
7,DJf3RQg2z2mqLMpceEjgCz,Robert Francisco
8,dtYFggVXRPULphFSCrqZqX,Kristen Parke
9,fBcAAEZ7NwMjqbuczyddDm,Brenda Bernstein


# INSERT USER_FRIEND 

In [21]:
# getting the ids
cur.execute('SELECT user_id FROM USER;')
rows = cur.fetchall()
df_user_id = pd.DataFrame(data=rows, columns=['user_id'])

In [29]:
import random
list_user_id = df_user_id.user_id
for user_id in list_user_id:
    l = list_user_id.tolist()
    l.remove(user_id)
    p = random.sample(l, 3)
    # print(p)
    for u in p: 
        print(user_id, u)
        print(u, user_id)
        cur.execute(
        "INSERT IGNORE INTO USER_FRIEND (user_id, friend) VALUES (?, ?);", 
        (user_id, u))
        cur.execute(
        "INSERT IGNORE INTO USER_FRIEND (user_id, friend) VALUES (?, ?);", 
        (u, user_id))

3xyaufSzzUp9LPkSKTxhqz HBxh8rpB33ymAsymLqutaC
HBxh8rpB33ymAsymLqutaC 3xyaufSzzUp9LPkSKTxhqz
3xyaufSzzUp9LPkSKTxhqz jzKtPvwCjUivKCTuu3jbWs
jzKtPvwCjUivKCTuu3jbWs 3xyaufSzzUp9LPkSKTxhqz
3xyaufSzzUp9LPkSKTxhqz fBcAAEZ7NwMjqbuczyddDm
fBcAAEZ7NwMjqbuczyddDm 3xyaufSzzUp9LPkSKTxhqz
3xyaufSzzUp9LPkSKTxhqz jPY9S3qZxfF9CUfda9FEBT
jPY9S3qZxfF9CUfda9FEBT 3xyaufSzzUp9LPkSKTxhqz
3xyaufSzzUp9LPkSKTxhqz NAUreDN99eJRFq3hdsR6Eh
NAUreDN99eJRFq3hdsR6Eh 3xyaufSzzUp9LPkSKTxhqz
5jZkqyec54QYyaqgp4h47u HqwTnrNdWnYz4XR9yPSPQ9
HqwTnrNdWnYz4XR9yPSPQ9 5jZkqyec54QYyaqgp4h47u
5jZkqyec54QYyaqgp4h47u nCnJGmEeaSNV2enHDrjFdh
nCnJGmEeaSNV2enHDrjFdh 5jZkqyec54QYyaqgp4h47u
5jZkqyec54QYyaqgp4h47u QYvpXBbuMfqxHTTb8jCoou
QYvpXBbuMfqxHTTb8jCoou 5jZkqyec54QYyaqgp4h47u
5jZkqyec54QYyaqgp4h47u fBcAAEZ7NwMjqbuczyddDm
fBcAAEZ7NwMjqbuczyddDm 5jZkqyec54QYyaqgp4h47u
5jZkqyec54QYyaqgp4h47u CbE6U4QrKJTcnUCMnW5FiZ
CbE6U4QrKJTcnUCMnW5FiZ 5jZkqyec54QYyaqgp4h47u
5qqCExUuQ5drJaYt7F3B5K i4r94zgoVD4owXoMut5LSY
i4r94zgoVD4owXoMut5LSY 5qqCExUuQ5d

In [ ]:
cur.execute("SELECT * FROM USER_FRIEND;")
rows = cur.fetchall()
df_user = pd.DataFrame(data=rows, columns=['user_id', 'friend'])
df_user

# INSERT HAS_GROUP, USER_HAS_GROUP_EXPENSE, EXPENSE 

In [30]:
# getting the ids
cur.execute('SELECT user_id FROM USER;')
rows = cur.fetchall()
df_user_id = pd.DataFrame(data=rows, columns=['user_id']).user_id.tolist()

In [10]:
from faker import Faker
import datetime
fake = Faker()

exp_date = fake.date_time_between(start_date='-1y', end_date='now').strftime('%Y-%m-%d')
exp_date

'2023-02-16'

In [31]:
from mariadb.constants import *

groupno=1
for user_id in df_user_id:
    cur.execute("SELECT friend FROM USER_FRIEND where user_id= ? ;",(user_id,)) # getting the friends of the user
    rows = cur.fetchall()
    df_user_id = pd.DataFrame(data=rows, columns=['user_id']).user_id.tolist() # getting the list

    # random generating of group expsense
    x=random.randrange(1,10)
    for i in range(x):
        group_id = shortuuid.uuid()     
        
        samples  = random.randrange(1,5) # for getting the number of debtor 
        p = random.sample(df_user_id, samples) # getting the random debtors
          
        amount = random.randrange(100,1000)
        group_name =  'Group {}'.format(groupno) # if (randombool) return null else group no.
        groupno+=1
        
        # INSERT HAS_GROUP
        cur.execute("INSERT INTO HAS_GROUP VALUES (? , ? );", (group_id, group_name)) 

        # INSERT USER_HAS_GROUP_EXPENSE
        exp_date = fake.date_time_between(start_date='-1y', end_date='now').strftime('%Y-%m-%d')
        
        expense_id = shortuuid.uuid() # expense id of the creditor
        cur.execute("INSERT INTO USER_HAS_GROUP_EXPENSE VALUES (? , ? , ?);", (user_id, group_id, expense_id)) 
        cur.execute("INSERT INTO EXPENSE VALUES (?, ?, ?, ?, ?)", (expense_id, user_id, amount, 1,exp_date)) # inserting expense

        for debtor in p:
            expense_id = shortuuid.uuid() # expense id of the debtop
            cur.execute("INSERT INTO USER_HAS_GROUP_EXPENSE VALUES (? , ? , ?);", (debtor, group_id, expense_id)) 
            cur.execute("INSERT INTO EXPENSE VALUES (?, ?, ?, ?, ?)", (expense_id, user_id, amount, 0, exp_date)) 

        

In [32]:
conn.commit()

In [13]:
# TRUNCATING THE tables
# cur.execute("TRUNCATE TABLE USER;")
# cur.execute("TRUNCATE TABLE USER_FRIEND;")
# cur.execute("ALTER TABLE TRUNCATE TABLE HAS_GROUP;")


In [14]:
# free resources
# cur.close()
# conn.close()